In [2]:
%run "../01-check_setup.ipynb"

SAP HANA Client for Python: 2.23.25021400
Connected to SAP HANA db version 4.00.000.00.1739875052 (fa/CE2024.40) 
at c5889dd5-e0f6-4930-8408-94d53ca61dbf.hna0.prod-us10.hanacloud.ondemand.com:443 as CODEJAMHANAAI00
Current time on the SAP HANA server: 2025-02-26 21:46:03.835000


In [3]:
import requests, os

os.environ["USER_AGENT"] = f"{requests.get('https://httpbin.org/user-agent').json()['user-agent']}"
os.environ["USER_AGENT"]

'python-requests/2.32.3'

In [4]:
import zipfile
import requests
from io import BytesIO

# URL of the zipped file
zip_url = "https://docs.python.org/3/archives/python-3.13-docs-text.zip"

# Download the zipped file
response = requests.get(zip_url)
zip_file = BytesIO(response.content)


In [5]:

# Load documents from the zipped file
documents_from_zip = []
with zipfile.ZipFile(zip_file, 'r') as z:
    for file_name in z.namelist():
        if not z.getinfo(file_name).is_dir():
            with z.open(file_name) as f:
                documents_from_zip.append({'metadata': {'file_name': file_name}, 'content': f.read().decode('utf-8')})

# Check the number of documents loaded
print(f"Number of documents loaded: {len(documents_from_zip)}")

Number of documents loaded: 507


In [6]:
## Select only records where file_name contains '/faq/'
faq_documents = [doc for doc in documents_from_zip if '/faq/' in doc['metadata']['file_name']]  
print(f"Number of FAQ documents: {len(faq_documents)}")

Number of FAQ documents: 9


In [15]:
print(faq_documents[2]['content'])

Library and Extension FAQ
*************************


General Library Questions


How do I find a module or application to perform task X?
--------------------------------------------------------

Check the Library Reference to see if there's a relevant standard
library module.  (Eventually you'll learn what's in the standard
library and will be able to skip this step.)

For third-party packages, search the Python Package Index or try
Google or another web search engine.  Searching for "Python" plus a
keyword or two for your topic of interest will usually find something
helpful.


Where is the math.py (socket.py, regex.py, etc.) source file?
-------------------------------------------------------------

If you can't find a source file for a module it may be a built-in or
dynamically loaded module implemented in C, C++ or other compiled
language. In this case you may not have the source file or it may be
something like "mathmodule.c", somewhere in a C source directory (not
on the Python

In [8]:
import pandas as pd

## Convert faq_documents to a DataFrame `df_faq`
df_faq = pd.DataFrame(faq_documents)


In [17]:
import markdown
from IPython.display import HTML

# Sample Markdown text
markdown_text = "# Header\n\nThis is a **bold** text and this is *italic*."

# Convert Markdown to HTML
html = markdown.markdown(df_faq['content'][0])

display((html))


'<p>"Why is Python Installed on my Computer?" FAQ</p>\n<hr />\n<h1>What is Python?</h1>\n<p>Python is a programming language.  It\'s used for many different\napplications. It\'s used in some high schools and colleges as an\nintroductory programming language because Python is easy to learn, but\nit\'s also used by professional software developers at places such as\nGoogle, NASA, and Lucasfilm Ltd.</p>\n<p>If you wish to learn more about Python, start with the Beginner\'s\nGuide to Python.</p>\n<h1>Why is Python installed on my machine?</h1>\n<p>If you find Python installed on your system but don\'t remember\ninstalling it, there are several possible ways it could have gotten\nthere.</p>\n<ul>\n<li>\n<p>Perhaps another user on the computer wanted to learn programming and\n  installed it; you\'ll have to figure out who\'s been using the machine\n  and might have installed it.</p>\n</li>\n<li>\n<p>A third-party application installed on the machine might have been\n  written in Python and i

In [18]:
# Convert column metadata of df_faq to string
df_faq['metadata'] = df_faq['metadata'].astype(str)
df_faq['content_html'] = df_faq['content'].apply(markdown.markdown)

In [19]:
display(df_faq.dtypes)
df_faq=df_faq.convert_dtypes()
display(df_faq.dtypes)

metadata        object
content         object
content_html    object
dtype: object

metadata        string[python]
content         string[python]
content_html    string[python]
dtype: object

In [20]:
hana_table_name="FAQ_HTML"

In [21]:
## Create a table in SAP HANA from `df_faq`
from hana_ml.dataframe import create_dataframe_from_pandas
hdf_faq_bronze = create_dataframe_from_pandas(connection_context=myconn, 
                                              pandas_df=df_faq, 
                                              table_name=hana_table_name, 
                                              force=True,
                                              object_type_as_bin=True,
                                              table_structure={'metadata': 'NVARCHAR(5000)' ,
                                                               'content': 'NCLOB', 'content_html': 'NCLOB'}
                                              )

100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


In [25]:
import pandas as pd
pd.set_option('max_colwidth', 256) 

hdf_faq_bronze.head().collect().T

,0
metadata,{'file_name': 'python-3.13-docs-text/faq/installed.txt'}
content,"""Why is Python Installed on my Computer?"" FAQ\n*********************************************\n\n\nWhat is Python?\n===============\n\nPython is a programming language. It's used for many different\napplications. It's used in some high schools and coll..."
content_html,"<p>""Why is Python Installed on my Computer?"" FAQ</p>\n<hr />\n<h1>What is Python?</h1>\n<p>Python is a programming language. It's used for many different\napplications. It's used in some high schools and colleges as an\nintroductory programming langua..."


## Text splitting, preparing FAQs for vectorization

In [26]:
hdf_faq_bronze.add_id().head().collect().T

,0
ID,1
metadata,{'file_name': 'python-3.13-docs-text/faq/installed.txt'}
content,"""Why is Python Installed on my Computer?"" FAQ\n*********************************************\n\n\nWhat is Python?\n===============\n\nPython is a programming language. It's used for many different\napplications. It's used in some high schools and coll..."
content_html,"<p>""Why is Python Installed on my Computer?"" FAQ</p>\n<hr />\n<h1>What is Python?</h1>\n<p>Python is a programming language. It's used for many different\napplications. It's used in some high schools and colleges as an\nintroductory programming langua..."


In [32]:
# Applying the Text Splitter with recursive-splitting, available with hana-ml 2.23
from hana_ml.text.text_splitter import TextSplitter

splitter = TextSplitter(split_type='recursive', doc_type='html')
splitter._extend_pal_parameter({'GLOBAL_SEPARATOR':'[<h]', 'KEEP_SEPARATOR':1})
hdf_faq_silver = splitter.split_text(
    hdf_faq_bronze.add_id().select('ID', 'content_html'), 
    order_status=True
    )

In [52]:
# Applying the Text Splitter with recursive-splitting, available with hana-ml 2.23
from hana_ml.text.text_splitter import TextSplitter

splitter = TextSplitter(split_type='document', doc_type='html', keep_separator=True, overlap=4)
splitter._extend_pal_parameter({'GLOBAL_SEPARATOR':'[<h1>,<h2>,<h3>,<h4>,<h5>,<h6>]'})
hdf_faq_silver = splitter.split_text(
    hdf_faq_bronze.add_id().select('ID', 'content_html'), 
    order_status=True
    )

In [53]:
import pandas as pd
pd.set_option('max_colwidth', None) 

print(hdf_faq_silver.shape)
display(splitter.statistics_.collect())
display(hdf_faq_silver.select("*", ('LENGTH("CONTENT")', "CHUNCK_SIZE")).head(10).collect())

[205, 3]


,STAT_NAME,STAT_VALUE
0,GLOBAL_SEPARATOR_LIST,"{""Separator"":""[<h1>,<h2>,<h3>,<h4>,<h5>,<h6>]""}"


,ID,SUB_ID,CONTENT,CHUNCK_SIZE
0,1,0,"<p>""Why is Python Installed on my Computer?"" FAQ</p>\n<hr />\n<h1>",64
1,1,1,"What is Python?</h1>\n<p>Python is a programming language. It's used for many different\napplications. It's used in some high schools and colleges as an\nintroductory programming language because Python is easy to learn, but\nit's also used by professional software developers at places such as\nGoogle, NASA, and Lucasfilm Ltd.</p>\n<p>If you wish to learn more about Python, start with the Beginner's\nGuide to Python.</p>\n<h1>",423
2,1,2,"Why is Python installed on my machine?</h1>\n<p>If you find Python installed on your system but don't remember\ninstalling it, there are several possible ways it could have gotten\nthere.</p>\n<ul>\n<li>\n<p>Perhaps another user on the computer wanted to learn programming and\n installed it; you'll have to figure out who's been using the machine\n and might have installed it.</p>\n</li>\n<li>\n<p>A third-party application installed on the machine might have been\n written in Python and included a Python installation. There are\n many such applications, from GUI programs to network servers and\n administrative scripts.</p>\n</li>\n<li>\n<p>Some Windows machines also have Python installed. At this writing\n we're aware of computers from Hewlett-Packard and Compaq that\n include Python. Apparently some of HP/Compaq's administrative tools\n are written in Python.</p>\n</li>\n<li>\n<p>Many Unix-compatible operating systems, such as macOS and some Linux\n distributions, have Python installed by default; it's included in\n the base installation.</p>\n</li>\n</ul>\n<h1>",1063
3,1,3,"Can I delete Python?</h1>\n<p>That depends on where Python came from.</p>\n<p>If someone installed it deliberately, you can remove it without\nhurting anything. On Windows, use the Add/Remove Programs icon in the\nControl Panel.</p>\n<p>If Python was installed by a third-party application, you can also\nremove it, but that application will no longer work. You should use\nthat application's uninstaller rather than removing Python directly.</p>\n<p>If Python came with your operating system, removing it is not\nrecommended. If you remove it, whatever tools were written in Python\nwill no longer run, and some of them might be important to you.\nReinstalling the whole system would then be required to fix things\nagain.</p>",718
4,2,4,<p>Python on Windows FAQ</p>\n<hr />\n<h1>,40
5,2,5,"How do I run a Python program under Windows?</h1>\n<p>This is not necessarily a straightforward question. If you are already\nfamiliar with running programs from the Windows command line then\neverything will seem obvious; otherwise, you might need a little more\nguidance.</p>\n<p>Unless you use some sort of integrated development environment, you\nwill end up <em>typing</em> Windows commands into what is referred to as a\n""Command prompt window"". Usually you can create such a window from\nyour search bar by searching for ""cmd"". You should be able to\nrecognize when you have started such a window because you will see a\nWindows ""command prompt"", which usually looks like this:</p>\n<p>C:&gt;</p>\n<p>The letter may be different, and there might be other things after it,\nso you might just as easily see something like:</p>\n<p>D:\YourName\Projects\Python&gt;</p>\n<p>depending on how your computer has been set up and what else you have\nrecently done with it. Once you have started such a window, you are\nwell on the way to running Python programs.</p>\n<p>You need to realize that your Python scripts have to be processed by\nanother program called the Python <em>interpreter</em>. The interpreter\nreads your script, compiles it into bytecodes, and then executes the\nbytecodes to run your program. So, how do you arrange for the\ninterpreter to handle your Python?</p>\n<p>First, you need to make sure that your command window recognises the\nword ""py"" as an instruction to start the interpreter. If you have\nopened a command window

## Generating Text Embeddings in SAP HANA Cloud

In [56]:
content_column = 'CONTENT'

In [55]:
print(f"""Number of records selected for further processing: {hdf_faq_silver.count()}""")

Number of records selected for further processing: 205


In [60]:
hdf_faq_silver.get_table_structure()

{'ID': 'INT', 'SUB_ID': 'INT', 'CONTENT': 'NCLOB'}

In [61]:
### Generating Text Embeddings in SAP HANA Cloud with the new PAL function, function available with hana-ml 2.23.
from hana_ml.text.pal_embeddings import PALEmbeddings
pe = PALEmbeddings()
hdf_faq_gold = pe.fit_transform(hdf_faq_silver, key="SUB_ID", target=[f"{content_column}"], thread_number=10, batch_size=10) #, max_token_num=512
print(f"{hdf_faq_gold.count()} records processed in {round(pe.runtime, 3)} sec")

205 records processed in 10.828 sec


In [62]:
hdf_faq_gold.get_table_structure()

{'ID': 'INT',
 'SUB_ID': 'INT',
 'CONTENT': 'NCLOB',
 'VECTOR_COL_CONTENT': 'REAL_VECTOR'}

In [64]:
hdf_faq_gold.head(2).collect()

,ID,SUB_ID,CONTENT,VECTOR_COL_CONTENT
0,1,0,"<p>""Why is Python Installed on my Computer?"" FAQ</p>\n<hr />\n<h1>","[0.033143430948257446, 0.030311258509755135, -0.006247238721698523, -0.016939878463745117, 0.045132335275411606, 0.004507202189415693, 0.0017255013808608055, 0.03091788850724697, -0.00022198859369382262, -0.03345530852675438, -0.02756786160171032, 0.018464984372258186, -0.07268993556499481, 0.019762547686696053, -0.003106691874563694, 0.03331155702471733, 0.07276413589715958, -0.01611221954226494, -0.004753657151013613, 0.04678535461425781, -0.03055003471672535, 0.04742305353283882, -0.06467708200216293, 0.007455017417669296, -0.06081252545118332, 0.004512310028076172, -0.0006425511674024165, 0.007020565681159496, -0.057269200682640076, -0.05362165346741676, 0.051400188356637955, 0.014546925202012062, 0.0010902599897235632, -0.036813925951719284, 0.02332942560315132, -0.0015924132894724607, -0.031235583126544952, 0.03840479254722595, -0.03235818073153496, 0.012237551622092724, 3.0626745228801155e-06, -0.01173030398786068, -0.01262391172349453, 0.043833743780851364, -0.059163898229599, 0.02255098894238472, -0.010658904910087585, 0.038852568715810776, 0.003964002709835768, 0.011811654083430767, -0.023250121623277664, 0.04976923018693924, -0.015482660382986069, -0.0678335577249527, 0.025107253342866898, 0.029572248458862305, 0.00831320509314537, -0.05473675951361656, -0.013811982236802578, -0.006573001854121685, 0.03987565264105797, 0.01776598021388054, -0.027233997359871864, 0.012870930135250092, 0.018448250368237495, -0.03847121819853783, 0.0010136663913726807, 0.06266427785158157, -0.029052501544356346, 0.023145483806729317, 0.021631749346852303, 0.038326237350702286, -0.06536207348108292, -0.027366865426301956, 0.03444402664899826, -0.004832293838262558, -0.016421547159552574, 0.05291067063808441, 0.030835695564746857, 0.04059726744890213, 0.020052867010235786, 0.03937257453799248, 0.037940021604299545, 0.05398465320467949, 0.027393177151679993, -0.014652387239038944, -0.04105612635612488, -0.0009165100054815412, -0.0017121442360803485, 0.04155111312866211, -0.0247059129178524, -0.03783412650227547, 0.04741296544671059, 0.0038822987116873264, -0.007127377670258284, -0.03903530538082123, 0.0031897309236228466, 0.006337149068713188, 0.02823573537170887, 0.00013904759543947875, ...]"
1,1,1,"What is Python?</h1>\n<p>Python is a programming language. It's used for many different\napplications. It's used in some high schools and colleges as an\nintroductory programming language because Python is easy to learn, but\nit's also used by professional software developers at places such as\nGoogle, NASA, and Lucasfilm Ltd.</p>\n<p>If you wish to learn more about Python, start with the Beginner's\nGuide to Python.</p>\n<h1>","[0.03812775760889053, 0.03766943886876106, -0.003967163152992725, -0.0042003849521279335, 0.022935397922992706, 0.006581943016499281, 0.00827502366155386, 0.07575546205043793, -0.035610876977443695, -0.026387322694063187, -0.05020380765199661, -0.0034398278221488, -0.06700544059276581, 0.03167089447379112, -0.005307564977556467, 0.022569259628653526, 0.10008034855127335, -0.010286636650562286, 0.011069213971495628, 0.03883926197886467, -0.05287667363882065, 0.015732107684016228, -0.02826469950377941, 0.0283032339066267, -0.01034550741314888, 0.014261618256568909, -0.022535227239131927, 0.011489572934806347, -0.05141935497522354, -0.03669305518269539, 0.016113653779029846, 0.010136461816728115, -0.012747722677886486, -0.0199532862752676, 0.011676655150949955, -0.022923801094293594, 0.00922334287315607, 0.01621725782752037, -0.020995136350393295, 0.008242526091635227, 0.013580135069787502, -0.039491601288318634, -0.03111039102077484, -0.018279066309332848, -0.058238085359334946, 0.007727416232228279, -0.004431350156664848, 0.04834586754441261, -0.02156693860888481, 0.028713161125779152, -0.031465914100408554, 0.01684660278260708, 0.008428304456174374, -0.05439024418592453, 0.003946194890886545, 0.033163521

In [65]:
hdf_faq_gold.select_statement

'SELECT T0."ID", T0."SUB_ID", T0."CONTENT", T1."VECTOR_COL_CONTENT"\nFROM (SELECT * FROM "#PAL_CHUNKING_RESULT_TBL_1E1607FC_F491_11EF_BDE1_9A1A5F2997FD") T0 INNER JOIN (SELECT "SUB_ID", "VECTOR_COL" AS "VECTOR_COL_CONTENT" FROM (SELECT "SUB_ID", "VECTOR_COL" FROM (SELECT * FROM "#PAL_EMBEDDINGS_RESULT_TBL_0_293DED88_F492_11EF_BDE1_9A1A5F2997FD") AS "DT_123") AS "DT_127") T1\n ON T0."SUB_ID" = T1."SUB_ID"\n'

In [66]:
hdf_faq_gold=hdf_faq_gold.save(where="#FAQ_EMBEDDINGS", force=True)

## Semantic search in FAQ

In [105]:
prompt="How to learn Python?"

In [106]:
df_result = myconn.sql(
    f"""SELECT TOP 5
    COSINE_SIMILARITY(VECTOR_EMBEDDING('{prompt}', 'QUERY', 'SAP_NEB.20240715'), "VECTOR_COL_{content_column}") AS "SIMILARITY",
    "ID", "{content_column}"
    FROM ({hdf_faq_gold.select_statement})
    ORDER BY 1 DESC;
    """
).collect()

In [107]:
df_result.head(5)

SIMILARITY  ID  \
0    0.794223   9   
1    0.736561   9   
2    0.724282   1   
3    0.711393   9   
4    0.710006   1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [102]:
# Print the rows of the 'content' column
print(df_result['CONTENT'][0])

Why was Python created in the first place?</h2>
<p>Here's a <em>very</em> brief summary of what started it all, written by Guido
van Rossum:</p>
<p>I had extensive experience with implementing an interpreted
   language in the ABC group at CWI, and from working with this group
   I had learned a lot about language design.  This is the origin of
   many Python features, including the use of indentation for
   statement grouping and the inclusion of very-high-level data types
   (although the details are all different in Python).</p>
<p>I had a number of gripes about the ABC language, but also liked
   many of its features.  It was impossible to extend the ABC language
   (or its implementation) to remedy my complaints -- in fact its lack
   of extensibility was one of its biggest problems.  I had some
   experience with using Modula-2+ and talked with the designers of
   Modula-3 and read the Modula-3 report. Modula-3 is the origin of
   the syntax and semantics used for exceptions, and

In [108]:
from IPython.display import HTML

# Convert the rows of the 'content' column to markdown format
display(HTML(df_result['CONTENT'][3]))